In [1]:
import numpy as np
import pandas as pd
import cv2
import os

numcam_data = pd.read_csv("D:/TRABAJOS RICARDO/Semestre_13/PFG/Repositorio/manual_annotations/CAMERA.csv", sep= ";")
position_data = pd.read_csv("D:/TRABAJOS RICARDO/Semestre_13/PFG/Repositorio/manual_annotations/DATA.csv", sep= ";")


In [2]:
camara_days = []
position_days = []

# separando el contenido del dataset entre los 3 dias
# primer día: 32 personas, segundo y tercero: 30 personas. Total 92 personas

sub_arrays = np.split(np.transpose(position_data.values), [32*7,62*7])
for i, _ in enumerate(sub_arrays):
    position_days.append(np.transpose(sub_arrays[i]))

sub_arrays = np.split(np.transpose(numcam_data.values), [32,62])
for i, _ in enumerate(sub_arrays):
    camara_days.append(np.transpose(sub_arrays[i]))

#print(camara_days[2].shape)
#print(position_days[2].shape)

In [23]:
# direcciones de entradas y salidas
input_url = "D:/TRABAJOS RICARDO/Semestre_13/PFG/Github/Entrenamiento-Maestro-Estudiante/Acondicionamiento imagenes/Frames/Entradas"
output_url = "D:/TRABAJOS RICARDO/Semestre_13/PFG/Github/Entrenamiento-Maestro-Estudiante/Acondicionamiento imagenes/Frames/Salidas/Day1/Cam1"


def data_yolo_format(filename, id_class, x1, x2, y1, y2):
    center_x = (x1 + (x2 - x1) / 2) / 960
    center_y = (y1 + (y2 - y1) / 2) / 540
    bbox_width = (x2 - x1)/960 
    bbox_height = (y2 - y1)/540

    if os.path.isfile(filename):
        with open(filename, 'a') as f:
            f.write("\n" + str(id_class) + " " + str(center_x) + " " + str(center_y) + " " + str(bbox_width) + " " + str(bbox_height))
 
    else: 
        with open(filename, 'w') as f:
            f.write(str(id_class) + " " + str(center_x) + " " + str(center_y) + " " + str(bbox_width) + " " + str(bbox_height))
    

# funcion que imprime las etiquetas de los videos
def process_dayx_camx_framex(day2process, cam2process, frame_step):

    # Directorio donde se guardarán los .txt
    dirbase = "D:/TRABAJOS RICARDO/Semestre_13/PFG/Github/Entrenamiento-Maestro-Estudiante/Acondicionamiento imagenes/Procesamiento de las anotaciones/Output/Day"
    dirname = dirbase + str(day2process) + "/Cam" +  str(cam2process) + "/"

    # Crea el directorio si aún no existe
    if not os.path.exists(dirname):
        os.makedirs(dirname)

    # seleccionando el día (1,2 o 3)
    day_cam_info = pd.DataFrame(camara_days[day2process-1])
    day_position_info = pd.DataFrame(position_days[day2process-1])

    # recorriendo los frames del día
    for frame_num, frame_content in enumerate(day_cam_info.values):
        
        # Nombre del archivo a crear
        filename = dirname + "frame" + str(frame_num) + "_day" + str(day2process) + "_cam" + str(cam2process) + ".txt"

        # salto entre frames a analizar
        if frame_num % frame_step == 0:
            
            # abriendo imagen del frame
            #img = cv2.imread(input_url + "/Day" + str(day2process) + "/Cam" + str(cam2process) + "/frame{:05d}.png".format(frame_num))

            # obteniendo la posicion de las personas durante este frame
            for actual_person, camara_num in enumerate(frame_content):
                
                # comparando si están en la cámara que interesa analizar
                if camara_num == cam2process:
                    
                    # obteniendo la posicion de la persona
                    positions = (day_position_info.values[frame_num,7*actual_person+1:7*actual_person+5])
                    #cv2.rectangle(img, (positions[0], positions[1]), (positions[2], positions[3]), (0, 255, 0), 2)
                    
                    data_yolo_format(filename=filename, id_class=0, x1=positions[0], y1=positions[1], x2=positions[2], y2=positions[3])
                    #print(frame_num, actual_person, camara_num, camara_num, positions)

            # dibujando la etiqueta
            #cv2.imwrite(os.path.join(output_url, 'frame{:05d}.png'.format(frame_num)), img)


day_list = [1, 2, 3]
cam_list = [1, 2, 3]

for nday in day_list:
    for ncam in cam_list:
        print(nday, ncam, "started")
        process_dayx_camx_framex(day2process=nday, cam2process=ncam, frame_step=20)
        
print("finished")



1 1 started
1 2 started
1 3 started
2 1 started
2 2 started
2 3 started
3 1 started
3 2 started
3 3 started
finished
